## OpenCV Task: Detecting and Classifying Traffic Signs
## Objective:
### Use OpenCV to detect and classify traffic signs in images.

## Task:
### ● Loading a data set
### ○ You can use, for example, the German Traffic Sign Recognition Benchmark (GTSRB) as a data set.
### ○ Load traffic sign images and their labels.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt 
import cv2
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from PIL import Image

In [2]:
df_train = pd.read_csv('./gtsrb/Train.csv')
df_train.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png


In [3]:
# Number of classes
n_classes = df_train['ClassId'].nunique()
n_classes

43

In [4]:
df_train.describe()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId
count,39209.000000,39209.000000,39209.000000,39209.000000,39209.000000,39209.000000,39209.000000
mean,50.835880,50.328930,5.999515,5.962381,45.197302,44.728379,15.788390
std,24.306933,23.115423,1.475493,1.385440,23.060157,21.971145,12.013238
min,25.000000,25.000000,0.000000,5.000000,20.000000,20.000000,0.000000
25%,35.000000,35.000000,5.000000,5.000000,29.000000,30.000000,5.000000
50%,43.000000,43.000000,6.000000,6.000000,38.000000,38.000000,12.000000
75%,58.000000,58.000000,6.000000,6.000000,53.000000,52.000000,25.000000
max,243.000000,225.000000,20.000000,20.000000,223.000000,205.000000,42.000000


In [5]:
# paths to images 
gtsrb_path = './gtsrb/'

In [6]:
def load_train_data(data_path, n_classes, target_size=(32, 32), test_size=0.2, random_state=42):
    data = []
    labels = []

    for i in range(n_classes):
        class_path = os.path.join(data_path, '{0}'.format(i))
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)

            # check if the file is an image
            if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                img = cv2.imread(image_path)

                # resize the image
                img = cv2.resize(img, target_size)

                data.append(img)
                labels.append(i)

    data = np.array(data)
    labels = np.array(labels)

    train_data, test_data, train_labels, test_labels = train_test_split(
        data, labels, test_size = test_size, random_state = random_state)

    return train_data, test_data, train_labels, test_labels

In [7]:
# load data and labels
train_data, test_data, train_labels, test_labels = load_train_data(os.path.join(gtsrb_path, 'Train'), n_classes)

### ● Image processing
### ○ Apply image preprocessing such as conversion to greyscale, normalisation, and maybe binarisation.
### ○ Use edge detection or other techniques to extract traffic signs from images.

In [8]:
class GTSRBDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]

        if isinstance(img, torch.Tensor):
            # tensor to PIL Image
            img = transforms.ToPILImage()(img)

        if self.transform:
            img = self.transform(img)

        # back to PyTorch tensor
        img = transforms.ToTensor()(img)

        return {'image': img, 'label': label}


In [9]:
def preprocess_image(img):
    img = cv2.convertScaleAbs(img) # to 8-bit
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGR to RGB

    # random rotation
    angle = np.random.uniform(-25, 25)
    rows, cols, _ = img_rgb.shape
    rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    img_rotated = cv2.warpAffine(img_rgb, rotation_matrix, (cols, rows))

    # random horizontal flip
    if np.random.rand() < 0.5:
        img_rotated = cv2.flip(img_rotated, 1)
    
    # random vertical flip
    if np.random.rand() < 0.5:
        img_rotated = cv2.flip(img_rotated, 0)

    # RGB back to BGR
    img_bgr = cv2.cvtColor(img_rotated, cv2.COLOR_RGB2BGR)
    
    # grayscale
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    
    # Gaussian blur
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    
    # normalize to the range [0, 1]
    normalized = (blurred - blurred.min()) / (blurred.max() - blurred.min())
    
    # brighten a image if nessessory 
    if np.mean(normalized) < 0.5:
        brightness_factor = np.random.uniform(1.0, 2.0) 
        normalized = np.clip(normalized * brightness_factor, 0.0, 1.0)

    tensor_image = torch.from_numpy(normalized).float()
    
    return tensor_image

In [10]:
preprocessed_train_data = [preprocess_image(image) for image in train_data]
preprocessed_test_data = [preprocess_image(image) for image in test_data]

In [11]:
train_dataset = GTSRBDataset(preprocessed_train_data, train_labels, transform = None)
test_dataset = GTSRBDataset(preprocessed_test_data, test_labels, transform = None)

In [12]:
batch_size = 225
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [13]:
import torch.nn as nn

class GTSRB_CNN(nn.Module):
    def __init__(self, num_channels, num_classes, dropout_prob=0.25):
        super(GTSRB_CNN, self).__init__()
        # 1 conv
        self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        self.bn1 = nn.BatchNorm2d(32)
        self.elu1 = nn.LeakyReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
        # 2 conv
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        self.bn2 = nn.BatchNorm2d(64)
        self.elu2 = nn.LeakyReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
        # 3 conv
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn3 = nn.BatchNorm2d(128)
        self.elu3 = nn.LeakyReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
        # 4 conv
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn4 = nn.BatchNorm2d(256)
        self.elu4 = nn.LeakyReLU()
        self.maxpool4 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
        # 1 fc
        self.fc1 = nn.Linear(in_features=256 * 2 * 2, out_features=256)
        self.elu5 = nn.LeakyReLU()
        self.dropout = nn.Dropout(p=dropout_prob)

        # Output layer
        self.fc2 = nn.Linear(in_features=256, out_features=num_classes)

    def forward(self, x):
        # 1 conv
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.elu1(x)
        x = self.maxpool1(x)
        # 2 conv
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.elu2(x)
        x = self.maxpool2(x)
        # 3 conv
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.elu3(x)
        x = self.maxpool3(x)
        # 4 conv
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.elu4(x)
        x = self.maxpool4(x)
        # 1 fc
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.elu5(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [14]:
num_channels = 1
num_classes = 43
model = GTSRB_CNN(num_channels=num_channels, num_classes = num_classes)

In [16]:
criterion = nn.CrossEntropyLoss()

In [22]:
optimizer = optim.Adam(model.parameters(),lr=0.0008)
scheduler = optim.lr_scheduler.LinearLR(optimizer,start_factor=1.0,end_factor=0.5,total_iters=10)
# best

In [24]:
def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs):
    model.train()

    for epoch in range(num_epochs):
        total_correct = 0
        total_samples = 0
        total_loss = 0.0

        for batch in train_loader:
            inputs, labels = batch['image'], batch['label']

            optimizer.zero_grad()
            outputs = model(inputs)
            labels = labels.long()

            # loss
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

            total_loss += loss.item()
            
        # update the learning rate
        scheduler.step()
        # accuracy and average loss for the epoch
        accuracy = total_correct / total_samples
        average_loss = total_loss / len(train_loader)

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f}, Accuracy: {accuracy * 100:.2f}%')


In [25]:
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in test_loader:
            inputs, labels = batch['image'], batch['label']
            labels = labels.long()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

In [26]:
train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs = 20)

Epoch [1/20], Loss: 2.2331, Accuracy: 36.15%
Epoch [2/20], Loss: 1.0776, Accuracy: 65.05%
Epoch [3/20], Loss: 0.7198, Accuracy: 76.68%
Epoch [4/20], Loss: 0.5411, Accuracy: 82.36%
Epoch [5/20], Loss: 0.4077, Accuracy: 86.51%
Epoch [6/20], Loss: 0.3267, Accuracy: 89.20%
Epoch [7/20], Loss: 0.2555, Accuracy: 91.64%
Epoch [8/20], Loss: 0.2149, Accuracy: 93.05%
Epoch [9/20], Loss: 0.1591, Accuracy: 95.03%
Epoch [10/20], Loss: 0.1246, Accuracy: 96.10%
Epoch [11/20], Loss: 0.0950, Accuracy: 97.22%
Epoch [12/20], Loss: 0.0806, Accuracy: 97.71%
Epoch [13/20], Loss: 0.0800, Accuracy: 97.70%
Epoch [14/20], Loss: 0.0654, Accuracy: 98.13%
Epoch [15/20], Loss: 0.0544, Accuracy: 98.47%
Epoch [16/20], Loss: 0.0497, Accuracy: 98.63%
Epoch [17/20], Loss: 0.0482, Accuracy: 98.61%
Epoch [18/20], Loss: 0.0410, Accuracy: 98.96%
Epoch [19/20], Loss: 0.0352, Accuracy: 99.08%
Epoch [20/20], Loss: 0.0404, Accuracy: 98.81%


In [27]:
test_model(model, test_loader)

Test Accuracy: 87.22%
